# 1. Analýza dat

---
---
Analýza dat bývá prvním krokem k řešení úlohy po získání dat a vždy by měla předcházet dalšímu jejich zpracování. 
V systémech s velkými daty, kde dochází k průběžnému sběru dat, se analýza dat stává úkolem, který je třeba provádět opakovaně, a je možné jej automatizovat.

Účely analýzy dat jsou:
 - zjištění základních charakteristik dat a jejich porozumění - velikost, rychlost, struktura, typy a rozsahy
 - porozumění veličinám, které data reprezentují, a vztahům mezi nimi
 - ujištění se o konzistenci dat, odhalení případných nedostatků, chyb a anomálií
 - (re)formulace úkolů/problémů, který pomocí dat chceme řešit
 - návrh nejlepších postupů zpracování dat včetně vhodných ML metod
 - návrh metod dalšího sběru, zpracování a experimentů

Analýza dat není samoúčelná a vždy zkoumá data vzhledem ke specifickému zadání, které definuje účel zpracování dat. 
--> Na jakém zadání budeme pracovat v našem tutoriálu?
Pro účely tutoriálu jsme vybrali datovou sadu Microsoft News Dataset (MIND). Důvodem tohoto výběru je: 
  1 Dostupnost sady (licenční podmínky dovolují její využití - zejména proto, že byla vyřešena její anonymizace)
  2 Blízkost sady k jednomu z hlavních úkolů řešených v rámci vyzkumu reklamních systémů společnosti Seznam.cz - doporučování obsahu.

Datová sada MIND je zaměřená na doporučování článků uživatelům - čtenářům zpravodajského portálu [msn.com](https://www.msn.com/en-us/feed). Jak již bylo popsáno v prezentaci, **úkolem doporučování** je vybrat pro daného uživatele další obsahové položky (články), které odpovídají uživatelově okamžitému zájmu a u kterých je vysoká šance na to, že si je uživatel přečte. Mírou toho, zda se doporučovací službě podařilo vybrat správný článek je to, zda si čtenář doporučený článek přečetl, nebo nikoliv - tedy zda kliknul na nabízený článek, nebo nekliknul.
--> Naším úkolem pro tento tutoriál tedy bude **navrhnout algoritmus, který provede vyhodnocení vhodnosti článku pro daného uživatele na základě dostupných dat**. Jinými slovy, náš cílový algoritmus by měl provést **predikci prokliku daného článku vzhledem k dostupným datům**.

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os

try:
    from google.colab import drive

    drive.mount('/content/gdrive')
    BASE_DIR = "/content/gdrive/MyDrive/itacademy2022"
    IN_COLAB = True
except:
    IN_COLAB = False
    BASE_DIR = ".."

MIND_DATA_SOURCE_DIR = "tmp/mind"
ORIGINAL_TRAIN_INPUT_DIR = os.path.join(BASE_DIR, MIND_DATA_SOURCE_DIR, "train/")
ORIGINAL_TEST_INPUT_DIR = os.path.join(BASE_DIR, MIND_DATA_SOURCE_DIR, "test/")
OUTPUT_DIR = os.path.join(BASE_DIR, "output")

In [ ]:
# check the data directories
! ls -l $ORIGINAL_TRAIN_INPUT_DIR

In [ ]:
! ls -l $ORIGINAL_TEST_INPUT_DIR

---
#### Q: Co víme o datech a o procesu, který je vyprodukoval?

---
Popis datové sady MIND je zde: https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md. Co jsou hlavní poznatky?   
- anonymizované logy chování čtenářů zpravodajského portálu [msn.com](https://www.msn.com/en-us/feed)
- náhodně vybraný vzorek 50000 čtenářů, kteří provedli alespoň 5 kliků během 6 týdnů sběru dat 12.10.-22.11.2019
- tréninková data: sekvence článků přečtených uživatelem za 4 první týdny sběru dat
- testovací data: poslední den 5. týdnu
- soubory v datovém adresáři:

 ```
 +------------------------+-----------------------------------------------------------------------------+
 | File Name              | Description                                                                 |
 +------------------------+-----------------------------------------------------------------------------+
 | behaviors.tsv          | The click histories and impression logs of users                            |
 | news.tsv               | The information of news articles                                            |
 | entity_embedding.vec   | The embeddings of entities in news extracted from knowledge graph           |
 | relation_embedding.vec | The embeddings of relations between entities extracted from knowledge graph |
 +------------------------+-----------------------------------------------------------------------------+
 ```
- behaviors.tsv - data o chování uživatele:
  - slateid - ID slejtu (sekvence, která předcházela zobrazení impresí)
  - userid - ID uživatele
  - time - čas zobrazení imprese ve formátu 'MM/DD/YYYY HH:MM:SS AM/PM'.
  - history - časová posloupnost prokliknutých článků (jejich ID) uživatelem PŘED zobrazením imprese
  - impressions - neuspořádaný seznam zobrazených článků s indikací prokliku
- news.tsv - data o článcích
  - News ID - ID článku
  - Category - kategorie článku
  - Subcategory - podkategorie článku
  - Title - titul článku
  - Abstract - abstrakt článku
  - URL - url
  - Title Entities - entity titulu 
  - Anstract Entities - entity abstraktu 
    - Label - jméno entity (Wikidata knwoledge graph)
    - Type - typ entity ve Wikidata
    - WikidataId - ID entity ve Wikidata
    - Confidence - konfidenční skóre zařazení entity
    - OccurrenceOffsets - offset výskytu v textu titulu, nebo abstraktu
    - SurfaceForms - jméno entity tak, jak se vyskytuje v textu

#### A: Nyní máme základní představu o původu dat a jejich struktuře.
---   

Bez ohledu na řešenou úlohu, prvním krokem datové analýzy je načtení dat, kontrola jejich validity a konzistence.   

---   
#### Q: Načteme data o chování uživatele `behaviors.tsv`. Odpovídají data tomu, co o nich víme z dokumentace? Jsou v nich nevalidní záznamy?

In [ ]:
# read and show the main train data set (behaviors.tsv - data on user's behavior)
behaviors_train = pd.read_csv(os.path.join(ORIGINAL_TRAIN_INPUT_DIR, 'behaviors.tsv'), sep='\t', names=['slateid', 'userid', 'time', 'history', 'impressions'])
behaviors_train.info()
behaviors_train

In [ ]:
# check rows for occurence of invalid numbers
behaviors_train.loc[behaviors_train.isna().any(axis=1)].head(4)

In [ ]:
# columns that we check for invalid values exclude 'history', where 'NaN' value may just indicate 'no history'
nanchk_cols = ['slateid', 'userid', 'time', 'impressions']
N_nan = behaviors_train.loc[behaviors_train[nanchk_cols].isna().any(axis=1)].shape[0]
print(f'There are {N_nan} rows with invalid values in any of {nanchk_cols} columns')

In [ ]:
# read and show the test data set (behaviors.tsv - data on user's behavior)
behaviors_test = pd.read_csv(os.path.join(ORIGINAL_TEST_INPUT_DIR, 'behaviors.tsv'), sep='\t', names=['slateid', 'userid', 'time', 'history', 'impressions'])
behaviors_test.info()
behaviors_test

In [ ]:
N_nan_test = behaviors_test.loc[behaviors_test[nanchk_cols].isna().any(axis=1)].shape[0]
print(f'There are {N_nan_test} rows with invalid values in any of {nanchk_cols} columns')

#### A: Struktura dat v `behaviors.tsv` odpovídá popisu v dokumentaci
---   
#### Q: Načteme data o článcích (`news.tsv`). Odpovídají data tomu, co o nich víme z dokumentace? Jsou v nich nevalidní záznamy?

In [ ]:
# read and show the traning news data
news_train = pd.read_csv(os.path.join(ORIGINAL_TRAIN_INPUT_DIR, 'news.tsv'), sep='\t', names=['newsid', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities'])
news_train.info()
news_train

In [ ]:
# read and show the test news data
news_test = pd.read_csv(os.path.join(ORIGINAL_TEST_INPUT_DIR, 'news.tsv'), sep='\t', names=['newsid', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities'])
news_test.info()
news_test

#### A: Struktura dat v `news.tsv` odpovídá popisu v dokumentaci
---  

Další kroky při analýze dat by měly směřovat k volbě featur a modelu pro zadanou úlohu, kterou je predikce prokliku článků.
Zopakujeme o co v úloze jde a co při analýze dat budeme cíleně hledat.
#### Predikce prokliku článků:
- Co známe?
    - sekvenci interakcí uživatele s portálem msn.com, tedy které články si v minulosti přečetl - 'history' a 'impressions' v tabulce `behaviors_*`
    - ID, kategorii, subkategorii, URL, nadpis, abstrakt všech článků v tabulce `news_*`
    - čas, kdy jsou mu články předkládány čtenáři (tedy přibližný čas, kdy se čtenář rozhoduje, zda si článek přečte - proklikne nebo nepřečne - neproklikne)
- Co chceme?
    - služba, která čtenáři nabídne to, co ho zajímá --TEDY-- metoda, která na základě dostupných dat nabídne k prokliku odkaz na článek na který si uživatel prevděpodobně klikne
- Jak vyhodnotíme úspěšnost?
    - lepší metoda výběru článků povede k vyšší proklikovosti nabízených odkazů
    - výběrem nabízených odkazů tedy chceme **maximalizovat pravděpodobnost prokliku** --> nejedná se o pouhou klasifikaci do tříd 'klikne', 'neklikne' (článek s nízkou pravděpodobností nezahodíme), ale spíše o klasifikaci se současným vyhodnocením pravděpodobnosti (vybíráme články s vyšší pravděpodobností prokliku, než ty ostatní)
- Co hledáme v datech?
    - čtenářovo rozhodování zda klikne, nebo neklikne závisí na mnoha faktorech, mnoho z nich v datech nejsou zachyceny (nebo dokonce nejsou vůbec zachytitelné)
    - některé datové položky se čtenářovým rozhodnutím ale souvisí - například čtenářovy interakce z minulosti se řídí jeho dlouhodobými (např. zájem o investování) a krátkodobými zájmy (např. okamžitý zájem o určitou kryptoměnu) --> tyto informace jsou zachytitelné a v datech je máme
    - při analýze chceme posoudit, které datové položky s čtenářovým proklikem souvisí a které nesouvisí
- Která data tedy budeme zkoumat?
    - proklik - nabývá hodnot 0 nebo 1 a je obsažen spolu s ID článku u každého článku, který byl čtenáři nabídnut ve sloupci 'impressions' tabulky v souboru behaviors.tsv 
      - je důležité rozumět rozložení kliků přes podmnožiny dat vymnezené na základě zkoumaných datových položek
      - info o čtenářích
        - někteří čtenáři budou klikat více, jiní méně
      - info o nabízených článcích (ID, kategorii, subkategorii, URL, titul, abstrakt)
        - některé články, jejich kategorie a podkategorie budou populárnější, než jiné
      - info o článcích prokliknutých uživatelem v minulosti
      - info o posloupnosti článků prokliknutých iživatelen před impresí
      - čas zobrazení imprese
        - někteří uživatelé můžou klikat více v časné odpoledne, jiní pozdě večer
---   
#### Q: Začneme zkoumat záznamy o interakcích jednotlivých uživatelů. Kolik interakcí máme pro každého uživatele?

In [ ]:
# What is the sequence of interactions of a user like?
behaviors_train['datetime'] = pd.to_datetime(behaviors_train['time'], infer_datetime_format=True)        # convert time string to datetime type, so the data can be sorted by the time
behaviors_train = behaviors_train.sort_values('datetime')                                                # sort by the date and time
slates4user_sorted = behaviors_train.groupby('userid').agg({'slateid': 'count'}).rename(columns={'slateid': 'count'}).sort_values('count', ascending=False)
uid0 = slates4user_sorted.index[0]   # select a user ID with highest number of slates
uid1 = slates4user_sorted.index[-1]  # select a user ID with lowest number of slates
N_uid0_hits = behaviors_train[behaviors_train['userid']==uid0].shape[0]
N_uid1_hits = behaviors_train[behaviors_train['userid']==uid1].shape[0]
print(f'User {uid0} had most interactions ({N_uid0_hits}), the user {uid1} had least ({N_uid1_hits}).')

#### A: Nejvíce interakcí jednotlivého uživatele je 62, nejméně 1.
---
#### Q: Jak vypadá sekvence interakcí jednoho uživatele?

In [ ]:
T_earliest = behaviors_train[behaviors_train['userid']==uid0]['time'].values[0]           # see the histories of the user with highest number of slates
T_latest = behaviors_train[behaviors_train['userid']==uid0]['time'].values[-1]
DT_delta = behaviors_train[behaviors_train['userid']==uid0]['datetime'].values[-1] - behaviors_train[behaviors_train['userid']==uid0]['datetime'].values[0]
days = DT_delta.astype('timedelta64[h]').astype('float')/np.timedelta64(1,'D').astype('timedelta64[h]').astype('float')
print(f'User {uid0} performed his {N_uid0_hits} interactions between {T_earliest} and {T_latest}, that is within {days:.01f} days')

In [ ]:
print(f'Histories of interactions recorded in the data for the user {uid0}:\n---')
print(behaviors_train[behaviors_train['userid']==uid0][['slateid', 'time', 'impressions', 'history']].values[:4])   # see the histories of the selected user
print('\n---')

#### A: Vybraný uživatel provedl svých 62 interakcí za cca 5 a půl dne. Zdá se ale, že interakce ve sloupci 'history' jsou totožné.
---
#### Q: Jsou opravdu interakce ve sloupci 'history' totožné u všech uživatelů? 

In [ ]:
print('Histories of the first and the last interactions recorded in the data:\n---')
print(behaviors_train[behaviors_train['userid']==uid0][['history']].values[[0,-1]])
sameornot = 'are identical!' if behaviors_train[behaviors_train['userid']==uid0][['history']].values[0][0]==behaviors_train[behaviors_train['userid']==uid0][['history']].values[-1][0] else 'do differ...'
print(f'---\nThese two histories {sameornot}\n---')

In [ ]:
N_max_unique_histories = behaviors_train.groupby('userid').agg({'history': pd.Series.nunique}).values.max()
print(f'There are at most {N_max_unique_histories.astype(int)} uniques histories for any user.')

#### A: Ověřili jsme, že historie jsou shodné pro všechny záznamy každého jednotlivého uživatele. Jaké to může mít dopady?   
- Jedním z dopadů je to, že historie interakcí uživatele, která je zaznamenána v datech je vlastně statická featura - nenese žádnou informaci o tom, jak se mění chování uživatele v čase. Dobré vědět.   
---
#### Q: Platí totéž i pro testovací datovou sadu?

In [ ]:
slates4user_sorted_test = behaviors_test.groupby('userid').agg({'slateid': 'count'}).rename(columns={'slateid': 'count'}).sort_values('count', ascending=False)
N_max_hits_test, N_min_hits_test = slates4user_sorted_test['count'].max(), slates4user_sorted_test['count'].min()
N_max_unique_histories_test = behaviors_test.groupby('userid').agg({'history': pd.Series.nunique}).values.max()
print(f'The range of interactions per user is {N_min_hits_test} ... {N_max_hits_test}.')
print(f'There are at most {N_max_unique_histories_test.astype(int)} uniques histories for any user in the test set.')

#### A: Ověřili jsme, že historie všech jednotlivých uživatelů jsou ve všech slejtech shodné i v testovací sadě.   
---
#### Q: Podívejme se na rozsah datové sady - kolik slejtů a uživatelů v datech máme?

In [ ]:
# how many slates in training set
N_slates = behaviors_train.shape[0]
# how many unique users
N_users = behaviors_train['userid'].unique().shape[0]
print(f'{N_slates} slates.')
print(f'{N_users} unique users.')

In [ ]:
# same for the test set
N_slates_test = behaviors_test.shape[0]
# how many unique users
N_users_test = behaviors_test['userid'].unique().shape[0]
print(f'{N_slates_test} slates.')
print(f'{N_users_test} unique users.')

#### A: V trénovací datové sadě máme 157k slejtů a 50k uživatelů. V testovací sadě 73k slejtů a 50k uživatelů. V testovací sadě je méně uživatelů, než v trénovací sadě. To může mít důležité dopady.
---
#### Q: Jak vypadají počty slejtů na uživatele?   

In [ ]:
# slates per user - train
N_max_slates = slates4user_sorted['count'].max()
N_min_slates = slates4user_sorted['count'].min()
ax = slates4user_sorted.plot(kind='hist', y=['count'], bins=range(1,N_max_slates+1), cumulative=False, density=True, histtype='step', figsize=(20, 6), title='histogram of slates per user', color='b')
slates4user_sorted.plot(kind='hist', y=['count'], bins=range(1,N_max_slates+1), cumulative=True, density=True, histtype='step', figsize=(20, 6), ax=ax, color='r')
print(f'Minimum and maximum slates per user: {N_min_slates}, {N_max_slates}')

In [ ]:
# slates per user - test
N_max_slates_test = slates4user_sorted_test['count'].max()
N_min_slates_test = slates4user_sorted_test['count'].min()
ax = slates4user_sorted_test.plot(kind='hist', y=['count'], bins=range(1, N_max_slates_test+1), cumulative=False, density=True, histtype='step', figsize=(20, 6), title='histogram of slates per user', color='b')
slates4user_sorted_test.plot(kind='hist', y=['count'], bins=range(1, N_max_slates_test+1), cumulative=True, density=True, histtype='step', ax=ax, color='r')
print(f'Minimum and maximum slates per user: {N_min_slates_test}, {N_max_slates_test}')

#### A: Nejvíce uživatelů má malý počet slejtů, s počtem slejtů počet uživatelů rychle klesá. Jen pro menšinu uživatelů (~10%) bude možné modelovat jejich interakce pomocí sekvence. Pro ostatní uživatele může model využívat pouze statické featury.  
---
#### Q: Kolik článků je obsaženo v datech? (Kolik unikátních článků v historii, v impresích, celkem).

In [ ]:
# how many impressions
N_imps = behaviors_train['impressions'].map(lambda x: x.split()).explode('impressions').shape[0]
# set of unique articles in impressions
all_imp_articles = set(behaviors_train['impressions'].map(lambda x: x.split()).explode('impressions').map(lambda x: x[:-2]).unique())
# set of unique articles in histories
all_histr_articles = set(behaviors_train['history'].dropna().map(lambda x: x.split()).explode('history').unique())
# set of all unique articles
all_articles = all_imp_articles | all_histr_articles
common_articles = all_imp_articles & all_histr_articles
# counts
N_imp_articles, N_histr_articles, N_articles, N_common_articles = len(all_imp_articles), len(all_histr_articles), len(all_articles), len(common_articles)
print(f'{N_imp_articles} unique articles among impressions.')
print(f'{N_histr_articles} unique articles in histories.')
print(f'{N_articles} unique articles total.')
print(f'{N_common_articles} articles in both - the history and the impressions.')

In [ ]:
list(common_articles)[:5], list(all_articles-common_articles)[:5]

#### A: 20k článků v impresích, 33k v historii, 51 celkem. Pouze 2k článků v historii také v impresích - velmi malý překryv. 
---
#### Q: Kolikrát byly články nabízeny k přečtení (kolik impresí na stejný článek) a kolikrát byly přečteny (kolik kliků na stejný článek)?

In [ ]:
# get all impressions along with the click information
impressions = ( behaviors_train
               .assign(imparray=lambda x: x.impressions.map(lambda xx: xx.split()))
               .explode('imparray')
               .drop(['impressions', 'history'], axis=1)
               .assign(clicked=lambda x: x.imparray.map(lambda xx: xx.split())))

behaviors_train['imparray'] = behaviors_train['impressions'].map(lambda x: x.split())
impressions = behaviors_train.explode('imparray').drop(['impressions', 'history'], axis=1)
impressions['clicked'] = impressions['imparray'].map(lambda x: int(x[-1]))
impressions['id'] = impressions['imparray'].map(lambda x: x[:-2])
impressions = impressions.drop(['imparray'], axis=1)
# calculate CTR (Click Through Rate) for each unique article
ctrs_all = impressions.groupby('id').agg({'clicked': ['sum', 'count']}).droplevel(0, axis=1)
# counts of article impressions and clicks
N_imp_articles_max = ctrs_all['count'].max()
N_clk_articles_max = ctrs_all['sum'].max()
print(f'{N_imp_articles_max} max impressions of a unique article.')
print(f'{N_clk_articles_max} max clicks on a unique article.')

In [ ]:
# calculate CTR per each unique articles
THR_MIN_IMPS_4_CTR = 30  # require minimum amount of impressions
ctrs_all['ctr'] = ctrs_all['sum'].values/ctrs_all['count'].values
ctr_max_all = ctrs_all['ctr'].max()
ctrs = ctrs_all[ctrs_all['count']>=THR_MIN_IMPS_4_CTR]
ctr_max = ctrs['ctr'].max()
print(f'{ctr_max:.03f} highest achieved article CTR for articles with enough impressions (min {THR_MIN_IMPS_4_CTR} impressions).')
print(f'{ctr_max_all:.03f} highest achieved article CTR.')

#### A: Maximální počet impresí článku je 22.5k, maximání počet kliků 4k, nejvyšší CTR článku je 0.5. 
---
#### Q: Jak vypadá rozložení impresí a kliků mezi články? Jaká je jejich proklikovost?

In [ ]:
def plot_hist(df, columns, N_bins, title):
    ax = df.plot(
        y=columns, bins=N_bins,
        kind='hist', histtype='step', cumulative=False, 
        figsize=(20, 6), title=title)
    df.plot(y=columns, bins=N_bins, kind='hist', histtype='step', cumulative=True, ax=ax)

In [ ]:
N_bins = 100
plot_hist(ctrs_all, ['count'], N_bins, 'article impressions histogram')
plot_hist(ctrs, ['count'], N_bins, f'article impressions histogram (min {THR_MIN_IMPS_4_CTR} impressions)')
plot_hist(ctrs, ['sum'], N_bins, f'article clicks histogram (min {THR_MIN_IMPS_4_CTR} impressions)')
plot_hist(ctrs, ['ctr'], N_bins, f'article CTR histogram (min {THR_MIN_IMPS_4_CTR} impressions)')

#### A: Úzká skupina článků má vysokou proklikovost (>20%), drtivá většina má proklikovost velmi nízkou (<5%).
---
#### Q: Jak vypadá rozložení impresí, kliků mezi uživatele? Jaká je jejich 'klikavost'?

In [ ]:
# calculate user clickrate
uclicks_all = impressions[['clicked', 'userid']].groupby('userid').agg(['sum', 'count']).droplevel(0, axis=1)
uclicks_all['ucr'] = uclicks_all['sum']/uclicks_all['count']
N_imp_user_max, N_imp_user_min = uclicks_all['count'].max(), uclicks_all['count'].min()
N_clk_user_max, N_clk_user_min = uclicks_all['sum'].max(), uclicks_all['sum'].min()
print(f'{N_imp_user_min}...{N_imp_user_max} range of impressions for any user.')
print(f'{N_clk_user_min}...{N_clk_user_max} range of clicks of any user.')

In [ ]:
# calculate clcik rate per each unique articles
THR_MIN_IMPS_4_USR = 10  # require minimum amount of impressions
uclicks_all['ucr'] = uclicks_all['sum'].values/uclicks_all['count'].values
ucr_max_all, ucr_min_all = uclicks_all['ucr'].max(), uclicks_all['ucr'].min()
uclicks = uclicks_all[uclicks_all['count']>=THR_MIN_IMPS_4_USR]
ucr_max, ucr_min = uclicks['ucr'].max(), uclicks['ucr'].min()
print(f'{ucr_min:.03f}...{ucr_max:.03f} range of user click rate for users with enough impressions (min {THR_MIN_IMPS_4_USR} impressions).')
print(f'{ucr_min_all:.03f}...{ucr_max_all:.03f} range of user user click rate.')

In [ ]:
N_bins = 100
plot_hist(uclicks_all, ['count'], N_bins, 'user impressions histogram')
plot_hist(uclicks, ['count'], N_bins, f'user impressions histogram (min {THR_MIN_IMPS_4_USR} impressions)')
plot_hist(uclicks, ['sum'], N_bins, f'user clicks histogram (min {THR_MIN_IMPS_4_USR} impressions)')
plot_hist(uclicks, ['ucr'], N_bins, f'user click rate histogram (min {THR_MIN_IMPS_4_USR} impressions)')

#### A: Úzká skupina uživatelů má vysokou 'klikavost' (>20%), většina ji má velmi nízkou (<5%).
---
#### Q: Jaké jsou statistiky kategorií a subkategorií?

In [ ]:
# how many categories, subcategories and articles
N_cats = len(news_train['category'].unique())
N_subcats = len(news_train['subcategory'].unique())
N_articles_news = len(news_train['newsid'].unique())
print(f'{N_cats} categories, {N_subcats} subcategories, {N_articles_news} unique articles (vs. {N_articles} unique articles in the behaviors table)')

In [ ]:
# how many subcategories and articles per any single category
cats = news_train.groupby('category').agg({'subcategory': ['nunique', 'unique'], 'newsid': 'nunique'})
cats.columns = ['subcat_cnt', 'subcats', 'article_cnt']
cats = cats.sort_values(['article_cnt', 'subcat_cnt'], ascending=False)
N_sub4cat_min, N_sub4cat_max = cats['subcat_cnt'].min(), cats['subcat_cnt'].max()
N_art4cat_min, N_art4cat_max = cats['article_cnt'].min(), cats['article_cnt'].max()
print(f'{N_sub4cat_min}...{N_sub4cat_max} subcategories per any single category')
print(f'{N_art4cat_min}...{N_art4cat_max} unique articles per any single category')
cats

In [ ]:
# how many articles per any single subcategory
subcats = news_train.groupby('subcategory').agg({'category': 'first', 'newsid': 'nunique'}).rename(columns={'newsid': 'article_cnt'}).sort_values('article_cnt', ascending=False)
N_art4subcat_min, N_art4subcat_max = subcats['article_cnt'].min(), subcats['article_cnt'].max()
print(f'{N_art4subcat_min}...{N_art4subcat_max} unique articles per any single subcategory')
subcats

#### A: 17 kategorií, 264 subkategorií, 51k článků. Velmi nerovnoměrné zastoupení článků a subkategorií v kategoriích a článků v subkategoriích.
---
#### Q: Jaká jsou zastoupení kategorií, subkategorií v trénovací množině?

In [ ]:
# function to count occurrences of articles, subcategories, categories
def count_occurences(df_behav, df_news, col):
    articles = ( df_behav[[col]]
                .dropna(axis=0)
                .assign(occarray=lambda x: x[col].map(lambda xx: xx.split(' ')))
                .drop([col], axis=1)
                .explode('occarray')
               )
    N_arts = articles.shape[0]
    articles = ( articles
                .assign(newsid=lambda x: x.occarray.map(lambda xx: xx.split('-')[0]))  # due to impressions (for history same as rename)
                .drop(['occarray'], axis=1)
                .groupby('newsid')
                .agg({'newsid': 'count'})           
                .rename( columns={'newsid': 'cnt'} )
                .reset_index()
                .join(df_news[['newsid', 'category', 'subcategory']].set_index('newsid'), on='newsid', how='left')
                .fillna({'subcategory': '-NA-', 'category': '-NA-'})
                .assign(relcount=lambda x: x.cnt.map(lambda xx: float(xx)/float(N_arts)))  
                .sort_values('cnt', ascending=False)
               )
    subcats = ( articles
               .groupby('subcategory')
               .agg({'cnt': 'sum', 'category': 'first'})
               .assign(relcount=lambda x: x.cnt.map(lambda xx: float(xx)/float(N_arts)))  
               .sort_values('cnt', ascending=False)
              )
    cats = ( subcats
            .groupby('category')
            .agg({'cnt': 'sum'})
            .assign(relcount=lambda x: x.cnt.map(lambda xx: float(xx)/float(N_arts)))  
            .sort_values('cnt', ascending=False)
           )
    return articles, subcats, cats, N_arts

In [ ]:
articles_hist, subcats_hist, cats_hist, N_arts_hist = count_occurences(behaviors_train, news_train, 'history')
articles_imps, subcats_imps, cats_imps, N_arts_imps = count_occurences(behaviors_train, news_train, 'impressions')

In [ ]:
cats_hist

In [ ]:
cats_imps

In [ ]:
subcats_hist

In [ ]:
subcats_imps

#### A: Relativní zastoupení kategorií a subkategorií v history vs. impressions se velmi liší - pro modelování predikce prokliku bude zřejmě lepší výskyty článků v obou kategoriích odlišovat.
---
#### Q: Jaká jsou zastoupení kategorií, subkategorií v testovací množině?

In [ ]:
articles_hist_test, subcats_hist_test, cats_hist_test, N_arts_hist_test = count_occurences(behaviors_test, news_test, 'history')
articles_imps_test, subcats_imps_test, cats_imps_test, N_arts_imps_test = count_occurences(behaviors_test, news_test, 'impressions')

In [ ]:
cats_hist_test

In [ ]:
cats_imps_test

In [ ]:
subcats_hist_test

In [ ]:
subcats_imps_test

#### A: Relativní zastoupení kategorií a subkategorií v history vs. impressions se velmi liší i v testovací sadě. 
---
#### Q: Jak se liší rozdělení dat v testovací i trénovací množině?

#### A: 
---
#### Q: Jak dobře data, která máme k dispozici predikují prokliky?